In [1]:
# imports for later usage
import json
import os
import plotly
import plotly.graph_objs as go
import random
import sys
import pickle
import csv
import openpyxl

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
import probanno
import re

In [41]:
likelihood_template.data

{u'rxn00594_c': {'complexes': [{'complex': 'cpx30659',
    'probability': 0.0,
    'type': 'CPLX_NOREPS'}],
  'gpr': '',
  'probability': 0.0,
  'reaction': u'rxn00594_c',
  'type': 'HASCOMPLEXES'},
 u'rxn01087_c': {'complexes': [{'complex': 'cpx32720',
    'probability': 0.0,
    'type': 'CPLX_NOREPS'}],
  'gpr': '',
  'probability': 0.0,
  'reaction': u'rxn01087_c',
  'type': 'HASCOMPLEXES'},
 u'rxn03234_c': {'complexes': [{'complex': 'cpx34554',
    'probability': 0.0,
    'type': 'CPLX_NOREPS'}],
  'gpr': '',
  'probability': 0.0,
  'reaction': u'rxn03234_c',
  'type': 'HASCOMPLEXES'},
 u'rxn01734_c': {'complexes': [{'complex': 'cpx33135',
    'probability': 0.0,
    'type': 'CPLX_NOREPS'}],
  'gpr': '',
  'probability': 0.0,
  'reaction': u'rxn01734_c',
  'type': 'HASCOMPLEXES'},
 u'rxn02282_c': {'complexes': [{'complex': 'cpx34518',
    'probability': 0.0,
    'type': 'CPLX_NOREPS'}],
  'gpr': '',
  'probability': 0.0,
  'reaction': u'rxn02282_c',
  'type': 'HASCOMPLEXES'},
 u'rx

In [3]:
with open("../Data/Filtered_lacto_genome_ids.data",'rb') as f:
    genome_ids_list = pickle.load(f)

In [5]:
len(genome_ids_list)

849

In [4]:
count = 1
pattern = re.compile('fig\|+[.\dpeg]+')
metabolic_features = []

for genome in genome_ids_list:
    try:
        print("Opening "+ genome + ", Number " + str(count) + "/" + str(len(genome_ids_list)))
        likelihoods = pickle.load(open("/home/bneubert/Lactobacillus/likelihoods/"+genome+".probs"))
        features = pickle.load(open('/home/bneubert/Lactobacillus/features/'+genome+'.feats'))
        metabolic_patric_ids = []
        for reaction, values in likelihoods.data.items():
                if(values['gpr'] != '' and values['probability'] > 0):
                    matches = re.findall(pattern,values['gpr'])
                    for match in matches:
                        if(match not in metabolic_patric_ids):
                            metabolic_patric_ids.append(unicode(match))
        
        for i in range(0, len(features)):
            # exclude features that do not have a pgfam_id
            if('pgfam_id' in features[i].keys()):
                if(features[i]['patric_id'] in metabolic_patric_ids):
                    temp = {}
                    temp['pgfam_id'] = features[i]['pgfam_id']
                    temp['patric_id'] = features[i]['patric_id']
                    temp['genome_id'] = features[i]['genome_id']
                    metabolic_features.append(temp)
        count += 1
    except:
        print("Genome " + genome + " failed")
        count += 1

Opening 713605.5, Number 1/849
Opening 888801.3, Number 2/849
Opening 525328.13, Number 3/849
Opening 525328.4, Number 4/849
Opening 879297.3, Number 5/849
Opening 879298.3, Number 6/849
Opening 879299.3, Number 7/849
Opening 879300.3, Number 8/849
Opening 879302.3, Number 9/849
Opening 879301.3, Number 10/849
Opening 879303.3, Number 11/849
Opening 879304.3, Number 12/849
Opening 879295.3, Number 13/849
Opening 879296.3, Number 14/849
Opening 928327.3, Number 15/849
Opening 928328.3, Number 16/849
Opening 349123.13, Number 17/849
Opening 927703.3, Number 18/849
Opening 525341.3, Number 19/849
Opening 557436.17, Number 20/849
Opening 557436.4, Number 21/849
Opening 1340495.3, Number 22/849
Opening 557433.4, Number 23/849
Opening 863368.3, Number 24/849
Opening 863369.3, Number 25/849
Opening 585517.3, Number 26/849
Opening 548485.3, Number 27/849
Opening 491077.3, Number 28/849
Opening 1598.347, Number 29/849
Opening 1598.346, Number 30/849
Opening 1598.345, Number 31/849
Opening 1598.

Opening 1226675.3, Number 252/849
Opening 1226677.3, Number 253/849
Opening 1226678.3, Number 254/849
Opening 1423717.5, Number 255/849
Opening 1423717.3, Number 256/849
Opening 1226679.3, Number 257/849
Opening 1226676.3, Number 258/849
Opening 1314884.3, Number 259/849
Opening 272621.13, Number 260/849
Opening 1579.81, Number 261/849
Opening 1579.28, Number 262/849
Opening 1579.56, Number 263/849
Opening 1579.80, Number 264/849
Opening 1579.85, Number 265/849
Opening 1579.38, Number 266/849
Opening 1403312.3, Number 267/849
Opening 1346615.3, Number 268/849
Opening 630527.3, Number 269/849
Opening 679196.3, Number 270/849
Opening 324831.13, Number 271/849
Opening 1185324.3, Number 272/849
Opening 1380360.3, Number 273/849
Opening 525326.3, Number 274/849
Opening 1334627.3, Number 275/849
Opening 559301.5, Number 276/849
Opening 575603.3, Number 277/849
Opening 1596.34, Number 278/849
Opening 1596.54, Number 279/849
Opening 1596.38, Number 280/849
Opening 1596.51, Number 281/849
Openi

Opening 1590.323, Number 501/849
Opening 1590.336, Number 502/849
Opening 1590.337, Number 503/849
Opening 1590.338, Number 504/849
Opening 1590.267, Number 505/849
Opening 1590.226, Number 506/849
Opening 1590.200, Number 507/849
Opening 1590.269, Number 508/849
Opening 1590.593, Number 509/849
Opening 1590.264, Number 510/849
Opening 1590.228, Number 511/849
Opening 1590.229, Number 512/849
Opening 1590.230, Number 513/849
Opening 1590.231, Number 514/849
Opening 1590.227, Number 515/849
Opening 1590.232, Number 516/849
Opening 1590.233, Number 517/849
Opening 1590.234, Number 518/849
Opening 1590.235, Number 519/849
Opening 1590.236, Number 520/849
Opening 1590.237, Number 521/849
Opening 1590.238, Number 522/849
Opening 1590.239, Number 523/849
Opening 1590.203, Number 524/849
Opening 1590.204, Number 525/849
Opening 1590.205, Number 526/849
Opening 1590.207, Number 527/849
Opening 1590.206, Number 528/849
Opening 1590.208, Number 529/849
Opening 1590.209, Number 530/849
Opening 15

Opening 47715.159, Number 748/849
Opening 47715.158, Number 749/849
Opening 47715.156, Number 750/849
Opening 47715.140, Number 751/849
Opening 47715.139, Number 752/849
Opening 47715.171, Number 753/849
Opening 47715.151, Number 754/849
Opening 47715.279, Number 755/849
Opening 47715.278, Number 756/849
Opening 47715.155, Number 757/849
Opening 47715.172, Number 758/849
Opening 47715.281, Number 759/849
Opening 47715.280, Number 760/849
Opening 47715.154, Number 761/849
Opening 47715.153, Number 762/849
Opening 47715.152, Number 763/849
Opening 47715.138, Number 764/849
Opening 47715.150, Number 765/849
Opening 47715.330, Number 766/849
Opening 47715.277, Number 767/849
Opening 47715.137, Number 768/849
Opening 47715.276, Number 769/849
Opening 47715.244, Number 770/849
Opening 575599.3, Number 771/849
Opening 1381124.3, Number 772/849
Opening 525325.3, Number 773/849
Opening 712938.3, Number 774/849
Opening 767453.3, Number 775/849
Opening 334390.5, Number 776/849
Opening 1407052.3, 

In [5]:
feature_list = metabolic_features

In [14]:
features

[{u'aa_length': 197,
  u'aa_sequence': u'MGLTIYCYDPVIREKTLREFKSKYGISYKKLDNISNNKILNSNTRFYKEMLAASRKCKESGYIMNFTFSIGDLSYMDLRSYLANLMGIDFYFNERKRIQYECPEELINTPVMNFLLHSDKHGEFSAQELKDLLTQIEDKIPEKMKFSPIKIKRDEKYVIYMQNEDNRDIEYITTSFFKFLKATVANNLGWAYEQNNA',
  u'aa_sequence_md5': u'fc477e8141af1e496368f1319708f35e',
  u'accession': u'JRUT01000012',
  u'annotation': u'PATRIC',
  u'date_inserted': u'2015-03-14T07:55:11.600Z',
  u'date_modified': u'2015-03-14T07:55:11.600Z',
  u'end': 9344,
  u'feature_id': u'PATRIC.1579.22.JRUT01000012.CDS.8751.9344.fwd',
  u'feature_type': u'CDS',
  u'figfam_id': u'FIG00750642',
  u'genome_id': u'1579.22',
  u'genome_name': u'Lactobacillus acidophilus ATCC 4356',
  u'gi': 725540352,
  u'location': u'8751..9344',
  u'na_length': 594,
  u'na_sequence': u'ATGGGATTAACAATATACTGCTATGATCCAGTCATTAGAGAAAAAACGCTCAGAGAGTTTAAAAGTAAGTATGGTATATCTTATAAAAAATTAGATAACATTTCCAATAATAAAATTTTAAATAGTAATACCAGATTCTATAAAGAAATGCTGGCTGCTAGTCGCAAATGCAAAGAATCTGGCTATATAATGAATTTCACTTTTAGTATAGGTGATTTGAGC

In [6]:
# find all metabolic genes associated with reactions whose likelihood is greater than 1
metabolic_patric_ids = []
pattern = re.compile('fig\|+[.\dpeg]+')
for species in feature_files_species:
    temp = []
    for strain in species:
        try:
            likelihood_template = pickle.load(open("/home/bneubert/Lactobacillus/likelihoods/"+strain+".probs"))    
            for reaction, values in likelihood_template.data.items():
                if(values['gpr'] != '' and values['probability'] > 0):
                    matches = re.findall(pattern,values['gpr'])
                    for match in matches:
                        if(match not in temp):
                            temp.append(unicode(match))
        except:
            print(strain + " failed")
            
    metabolic_patric_ids.append(temp)

NameError: name 'feature_files_species' is not defined

In [45]:
unique_bois  = []
pattern = re.compile('\d+.{1}\d.')
count = 1
for species in metabolic_patric_ids:
    print(str(count) + " of 20")
    for feature in species:
        matches = re.findall(pattern,feature)
        
        if(matches[0] not in unique_bois):
            unique_bois.append(matches[0])
    count += 1
                
print(len(unique_bois))

1 of 20
2 of 20
3 of 20
4 of 20
5 of 20
6 of 20
7 of 20
8 of 20
9 of 20
10 of 20
11 of 20
12 of 20
13 of 20
14 of 20
15 of 20
16 of 20
17 of 20
18 of 20
19 of 20
20 of 20
656


In [6]:
to_del_upper = []
index = 0
for species in feature_list_species:
    index_in_features = 0
    to_del_inner = []
    for feature in species:
        if(feature['patric_id'] not in metabolic_patric_ids[index]):
            to_del_inner.append(index_in_features)
        
        index_in_features += 1
    to_del_upper.append(to_del_inner)
    index += 1
        

In [7]:
index = 0
for outer in to_del_upper:
    for i in reversed(outer):
        del feature_list_species[index][i]
    index += 1

In [3]:
feature_files = os.listdir('/home/bneubert/Lactobacillus/features')

In [29]:
# USE THIS FOR ANALYSIS OF 20 SPECIES
filename = '/home/bneubert/Lactobacillus/Data/PATRIC_genome_1508_with_meta_short_1023.xlsx'
wb = openpyxl.load_workbook(filename)
sheet = wb['patric3_query']

current = sheet.cell(row=2, column = 4).value
feature_files_species = []
temp = []

for i in range(2, 1025):
    if(sheet.cell(row=i, column = 4).value is current):
        temp.append(str(sheet.cell(row=i, column = 1).value))
    else:
        current = sheet.cell(row=i, column = 4).value
        feature_files_species.append(temp)
        temp = [str(sheet.cell(row=i, column = 1).value)]
feature_files_species.append(temp)

# delete bad model
del feature_files_species[0][feature_files_species[0].index('1579.60')]
#     print(i, sheet.cell(row=i,column=1).value)

feature_list_species = []
count = 0
for species in feature_files_species:
    temp_species = []
    for file in species:
        count += 1
        print("Opening: "+str(file)+", number "+str(count)+"")
        # load in the feature file as a pickle object, this will be a list
        feature = pickle.load(open('/home/bneubert/Lactobacillus/features/'+file+'.feats'))
        # add every feature to the total feature list for later use
        for i in range(0, len(feature)):
            # exclude features that do not have a pgfam_id
            if('pgfam_id' in feature[i].keys()):
                temp = {}
                temp['pgfam_id'] = feature[i]['pgfam_id']
                temp['patric_id'] = feature[i]['patric_id']
                temp['genome_id'] = feature[i]['genome_id']
                temp_species.append(temp)
    feature_list_species.append(temp_species)

Opening: 891391.3, number 1
Opening: 1579.22, number 2
Opening: 525306.3, number 3
Opening: 1421011.3, number 4
Opening: 1421011.4, number 5
Opening: 1226675.3, number 6
Opening: 1226677.3, number 7
Opening: 1226678.3, number 8
Opening: 1423717.5, number 9
Opening: 1423717.3, number 10
Opening: 1226679.3, number 11
Opening: 1226676.3, number 12
Opening: 1314884.3, number 13
Opening: 272621.13, number 14
Opening: 1579.81, number 15
Opening: 1579.28, number 16
Opening: 1579.49, number 17
Opening: 1579.56, number 18
Opening: 1579.80, number 19
Opening: 1579.33, number 20
Opening: 1579.85, number 21
Opening: 1579.38, number 22
Opening: 1423718.3, number 23
Opening: 1601.9, number 24
Opening: 1601.5, number 25
Opening: 1601.14, number 26
Opening: 1601.19, number 27
Opening: 1601.11, number 28
Opening: 1601.15, number 29
Opening: 1601.12, number 30
Opening: 1601.17, number 31
Opening: 1601.13, number 32
Opening: 1601.18, number 33
Opening: 1601.10, number 34
Opening: 1601.16, number 35
Openi

Opening: 1596.38, number 279
Opening: 1596.51, number 280
Opening: 1596.50, number 281
Opening: 1596.36, number 282
Opening: 575604.3, number 283
Opening: 1587.20, number 284
Opening: 1226336.3, number 285
Opening: 1226332.3, number 286
Opening: 1226333.3, number 287
Opening: 1226334.3, number 288
Opening: 1226335.3, number 289
Opening: 326425.4, number 290
Opening: 405566.6, number 291
Opening: 585520.4, number 292
Opening: 585520.12, number 293
Opening: 767462.3, number 294
Opening: 767456.3, number 295
Opening: 1587.21, number 296
Opening: 947827.8, number 297
Opening: 880633.7, number 298
Opening: 1587.24, number 299
Opening: 1587.53, number 300
Opening: 1587.25, number 301
Opening: 1587.50, number 302
Opening: 1587.42, number 303
Opening: 1587.22, number 304
Opening: 1587.33, number 305
Opening: 1587.34, number 306
Opening: 1587.32, number 307
Opening: 1587.159, number 308
Opening: 1587.23, number 309
Opening: 1587.86, number 310
Opening: 713605.5, number 311
Opening: 888801.3, nu

Opening: 1590.605, number 549
Opening: 1590.606, number 550
Opening: 1590.611, number 551
Opening: 1590.551, number 552
Opening: 1590.154, number 553
Opening: 1590.340, number 554
Opening: 1590.324, number 555
Opening: 1590.192, number 556
Opening: 1590.466, number 557
Opening: 1590.306, number 558
Opening: 1590.326, number 559
Opening: 1590.350, number 560
Opening: 1590.307, number 561
Opening: 1590.465, number 562
Opening: 1590.287, number 563
Opening: 1590.625, number 564
Opening: 1590.150, number 565
Opening: 1590.309, number 566
Opening: 1590.180, number 567
Opening: 1590.181, number 568
Opening: 1590.467, number 569
Opening: 1590.548, number 570
Opening: 1590.587, number 571
Opening: 1590.483, number 572
Opening: 1590.485, number 573
Opening: 1590.585, number 574
Opening: 1590.142, number 575
Opening: 1590.484, number 576
Opening: 1590.482, number 577
Opening: 1590.293, number 578
Opening: 1590.265, number 579
Opening: 1590.266, number 580
Opening: 1590.175, number 581
Opening: 1

Opening: 1390383.3, number 822
Opening: 568704.3, number 823
Opening: 525361.3, number 824
Opening: 1316933.3, number 825
Opening: 1318634.3, number 826
Opening: 1408205.4, number 827
Opening: 1203258.5, number 828
Opening: 1203259.4, number 829
Opening: 947828.9, number 830
Opening: 1457395.3, number 831
Opening: 1457396.3, number 832
Opening: 880592.5, number 833
Opening: 47715.87, number 834
Opening: 47715.101, number 835
Opening: 47715.102, number 836
Opening: 47715.67, number 837
Opening: 47715.88, number 838
Opening: 47715.100, number 839
Opening: 47715.105, number 840
Opening: 47715.103, number 841
Opening: 47715.106, number 842
Opening: 47715.99, number 843
Opening: 47715.288, number 844
Opening: 47715.104, number 845
Opening: 47715.107, number 846
Opening: 47715.109, number 847
Opening: 47715.110, number 848
Opening: 47715.111, number 849
Opening: 47715.112, number 850
Opening: 47715.108, number 851
Opening: 47715.113, number 852
Opening: 47715.114, number 853
Opening: 47715.1

In [314]:
# DO I NEED THIS STILL?
current = feature_list_species[0][0]['genome_id']
count = 0
covered_features = []

for feature in feature_list_species[0]:
    if (feature['genome_id'] == current):
        count += 1
        if(feature['pgfam_id'] not in covered_features):
            covered_features.append(feature['pgfam_id'])
    else:
        print('Non-unique ' + current + ' ' + str(count))
        print('Unique ' + current + ' ' + str(len(covered_features)))
        covered_features = [feature['pgfam_id']]
        count = 1
        current = feature['genome_id']
        

Non-unique 891391.3 2112
Unique 891391.3 1678
Non-unique 1579.22 1904
Unique 1579.22 1553
Non-unique 525306.3 1937
Unique 525306.3 1564
Non-unique 1421011.3 1122
Unique 1421011.3 958
Non-unique 1421011.4 1867
Unique 1421011.4 1553
Non-unique 1226675.3 1894
Unique 1226675.3 1557
Non-unique 1226677.3 1873
Unique 1226677.3 1552
Non-unique 1226678.3 1901
Unique 1226678.3 1563
Non-unique 1423717.5 1916
Unique 1423717.5 1563
Non-unique 1423717.3 1934
Unique 1423717.3 1572
Non-unique 1226679.3 1930
Unique 1226679.3 1587
Non-unique 1226676.3 1897
Unique 1226676.3 1562
Non-unique 1314884.3 1894
Unique 1314884.3 1562
Non-unique 272621.13 1882
Unique 272621.13 1546
Non-unique 1579.81 1942
Unique 1579.81 1594
Non-unique 1579.28 1908
Unique 1579.28 1580
Non-unique 1579.49 1903
Unique 1579.49 1555
Non-unique 1579.56 1947
Unique 1579.56 1581
Non-unique 1579.80 1917
Unique 1579.80 1584
Non-unique 1579.33 2039
Unique 1579.33 1582
Non-unique 1579.85 2079
Unique 1579.85 1707


In [10]:
# USE THIS IN CONJUNCTION WITH ABOVE FOR FULL SET OF 20 SPECIES
# AT ONCE
feature_list = []
for species in feature_list_species:
    for feature in species:
        feature_list.append(feature)

In [281]:
# feature_list

In [8]:
# USE THIS FOR FULL SET OF FEATURES

# exclude bad feature file due to low feature length(26), same genome id as the bad model. Only feature file below 500 feats
print(len(feature_files))
feature_files.remove("1579.60.feats")
# make sure deletion was successful
print(len(feature_files))
feature_list = []
counter = 0
for file in feature_files[0:len(feature_files)-1]:
    print("Opening: "+str(file))
    print("Current length: " + str(len(feature_list)))
    print("Current size: "+ str(sys.getsizeof(feature_list)))
    
    # load in the feature file as a pickle object, this will be a list
    feature = pickle.load(open('/home/bneubert/Lactobacillus/features/'+file))
    # add every feature to the total feature list for later use
    for i in range(0, len(feature)):
        # exclude features that do not have a pgfam_id
        if('pgfam_id' in feature[i].keys()):
            temp = {}
            temp['pgfam_id'] = feature[i]['pgfam_id']
            temp['patric_id'] = feature[i]['patric_id']
            temp['genome_id'] = feature[i]['genome_id']
            feature_list.append(temp)
    # limiter, will be removed to process full data set so as to not take too much time
#     counter += 1
#     if(counter is 200):
#         break


Opening: 1001583.3.feats
Current length: 0
Current size: 72
Opening: 1002365.5.feats
Current length: 2566
Current size: 21056
Opening: 1007676.4.feats
Current length: 5357
Current size: 43048
Opening: 1028490.4.feats
Current length: 7971
Current size: 69168
Opening: 1029822.3.feats
Current length: 11271
Current size: 98632
Opening: 1033837.3.feats
Current length: 13180
Current size: 111008
Opening: 1033983.3.feats
Current length: 15594
Current size: 124928
Opening: 1036177.3.feats
Current length: 17416
Current size: 140592
Opening: 1037411.3.feats
Current length: 20478
Current size: 178032
Opening: 1040964.3.feats
Current length: 22308
Current size: 200328
Opening: 1041521.3.feats
Current length: 24563
Current size: 200328
Opening: 1042399.3.feats
Current length: 26455
Current size: 225416
Opening: 1042400.3.feats
Current length: 28308
Current size: 253640
Opening: 1046596.4.feats
Current length: 30199
Current size: 253640
Opening: 1046596.6.feats
Current length: 32841
Current size: 28

Opening: 1256207.3.feats
Current length: 285337
Current size: 2380496
Opening: 1256208.3.feats
Current length: 288157
Current size: 2380496
Opening: 1256209.3.feats
Current length: 291085
Current size: 2380496
Opening: 1256210.3.feats
Current length: 294167
Current size: 2380496
Opening: 1256211.3.feats
Current length: 297207
Current size: 2380496
Opening: 1256212.3.feats
Current length: 300268
Current size: 2678104
Opening: 1256213.3.feats
Current length: 303444
Current size: 2678104
Opening: 1256214.3.feats
Current length: 306361
Current size: 2678104
Opening: 1256215.3.feats
Current length: 309362
Current size: 2678104
Opening: 1256216.3.feats
Current length: 312332
Current size: 2678104
Opening: 1256217.3.feats
Current length: 315430
Current size: 2678104
Opening: 1256218.3.feats
Current length: 318485
Current size: 2678104
Opening: 1256219.3.feats
Current length: 321407
Current size: 2678104
Opening: 1256220.3.feats
Current length: 324313
Current size: 2678104
Opening: 1256221.3.f

Opening: 1423722.3.feats
Current length: 598721
Current size: 4826320
Opening: 1423723.10.feats
Current length: 600341
Current size: 4826320
Opening: 1423723.12.feats
Current length: 602475
Current size: 4826320
Opening: 1423724.4.feats
Current length: 604756
Current size: 5429656
Opening: 1423725.3.feats
Current length: 606847
Current size: 5429656
Opening: 1423726.3.feats
Current length: 609189
Current size: 5429656
Opening: 1423727.3.feats
Current length: 612388
Current size: 5429656
Opening: 1423728.3.feats
Current length: 614322
Current size: 5429656
Opening: 1423729.3.feats
Current length: 616731
Current size: 5429656
Opening: 1423730.3.feats
Current length: 618593
Current size: 5429656
Opening: 1423730.4.feats
Current length: 622093
Current size: 5429656
Opening: 1423731.3.feats
Current length: 624621
Current size: 5429656
Opening: 1423732.4.feats
Current length: 626788
Current size: 5429656
Opening: 1423733.3.feats
Current length: 629706
Current size: 5429656
Opening: 1423733.4

Opening: 1423812.3.feats
Current length: 896573
Current size: 7731048
Opening: 1423813.3.feats
Current length: 899343
Current size: 7731048
Opening: 1423814.6.feats
Current length: 901879
Current size: 7731048
Opening: 1423815.3.feats
Current length: 903666
Current size: 7731048
Opening: 1423816.3.feats
Current length: 906043
Current size: 7731048
Opening: 1423817.3.feats
Current length: 909180
Current size: 7731048
Opening: 1423818.3.feats
Current length: 911698
Current size: 7731048
Opening: 1423819.3.feats
Current length: 914267
Current size: 7731048
Opening: 1423820.4.feats
Current length: 917128
Current size: 7731048
Opening: 1423821.4.feats
Current length: 918583
Current size: 7731048
Opening: 1423822.7.feats
Current length: 920194
Current size: 7731048
Opening: 1423822.8.feats
Current length: 923059
Current size: 7731048
Opening: 1423823.3.feats
Current length: 926154
Current size: 7731048
Opening: 1423823.4.feats
Current length: 928075
Current size: 7731048
Opening: 1423824.3.f

Opening: 1580.52.feats
Current length: 1171033
Current size: 9784704
Opening: 1580.53.feats
Current length: 1173850
Current size: 9784704
Opening: 1580.57.feats
Current length: 1176282
Current size: 9784704
Opening: 1580.58.feats
Current length: 1178733
Current size: 9784704
Opening: 1580.63.feats
Current length: 1181562
Current size: 9784704
Opening: 1580.64.feats
Current length: 1184068
Current size: 9784704
Opening: 1580.65.feats
Current length: 1186817
Current size: 9784704
Opening: 1580.66.feats
Current length: 1189693
Current size: 9784704
Opening: 1580.87.feats
Current length: 1192198
Current size: 9784704
Opening: 1580.90.feats
Current length: 1195096
Current size: 9784704
Opening: 1581051.3.feats
Current length: 1197906
Current size: 9784704
Opening: 1581062.3.feats
Current length: 1200623
Current size: 9784704
Opening: 1581063.3.feats
Current length: 1202525
Current size: 9784704
Opening: 1581136.3.feats
Current length: 1205432
Current size: 9784704
Opening: 1582.101.feats
Cu

...

Opening: 1613.133.feats
Current length: 2450188
Current size: 19836768
Opening: 1613.134.feats
Current length: 2452499
Current size: 19836768
Opening: 1613.136.feats
Current length: 2454442
Current size: 19836768
Opening: 1613.137.feats
Current length: 2456587
Current size: 19836768
Opening: 1613.138.feats
Current length: 2458724
Current size: 19836768
Opening: 1613.155.feats
Current length: 2460666
Current size: 19836768
Opening: 1613.156.feats
Current length: 2462783
Current size: 19836768
Opening: 1613.165.feats
Current length: 2464809
Current size: 19836768
Opening: 1613.166.feats
Current length: 2466951
Current size: 19836768
Opening: 1613.167.feats
Current length: 2469159
Current size: 19836768
Opening: 1613.168.feats
Current length: 2471385
Current size: 19836768
Opening: 1613.32.feats
Current length: 2473466
Current size: 19836768
Opening: 1613.34.feats
Current length: 2475658
Current size: 19836768
Opening: 1613.35.feats
Current length: 2478204
Current size: 19836768
Opening: 

Opening: 1926284.3.feats
Current length: 2706607
Current size: 22316408
Opening: 1946727.3.feats
Current length: 2708685
Current size: 22316408
Opening: 1946728.3.feats
Current length: 2709889
Current size: 22316408
Opening: 1946729.3.feats
Current length: 2711021
Current size: 22316408
Opening: 1946730.3.feats
Current length: 2712709
Current size: 22316408
Opening: 1946731.3.feats
Current length: 2713893
Current size: 22316408
Opening: 2042028.3.feats
Current length: 2714943
Current size: 22316408
Opening: 2042029.3.feats
Current length: 2716939
Current size: 22316408
Opening: 2042030.3.feats
Current length: 2719213
Current size: 22316408
Opening: 2042031.3.feats
Current length: 2721316
Current size: 22316408
Opening: 2042032.3.feats
Current length: 2723481
Current size: 22316408
Opening: 2042033.3.feats
Current length: 2725624
Current size: 22316408
Opening: 2042034.3.feats
Current length: 2727787
Current size: 22316408
Opening: 2042035.3.feats
Current length: 2729953
Current size: 2

Opening: 375175.85.feats
Current length: 2956771
Current size: 25106000
Opening: 387344.15.feats
Current length: 2959383
Current size: 25106000
Opening: 390333.12.feats
Current length: 2961654
Current size: 25106000
Opening: 390333.13.feats
Current length: 2964408
Current size: 25106000
Opening: 390333.7.feats
Current length: 2966350
Current size: 25106000
Opening: 392416.3.feats
Current length: 2968292
Current size: 25106000
Opening: 392416.4.feats
Current length: 2970506
Current size: 25106000
Opening: 396268.3.feats
Current length: 2972801
Current size: 25106000
Opening: 405566.6.feats
Current length: 2974321
Current size: 25106000
Opening: 417373.4.feats
Current length: 2976558
Current size: 25106000
Opening: 440496.3.feats
Current length: 2978160
Current size: 25106000
Opening: 440497.10.feats
Current length: 2980533
Current size: 25106000
Opening: 449659.4.feats
Current length: 2982117
Current size: 25106000
Opening: 47715.100.feats
Current length: 2984346
Current size: 25106000


Opening: 47770.28.feats
Current length: 3304236
Current size: 28244296
Opening: 47770.29.feats
Current length: 3306504
Current size: 28244296
Opening: 47770.30.feats
Current length: 3309096
Current size: 28244296
Opening: 47770.31.feats
Current length: 3311672
Current size: 28244296
Opening: 47770.32.feats
Current length: 3314222
Current size: 28244296
Opening: 47770.33.feats
Current length: 3316485
Current size: 28244296
Opening: 47770.34.feats
Current length: 3319048
Current size: 28244296
Opening: 47770.35.feats
Current length: 3321298
Current size: 28244296
Opening: 47770.36.feats
Current length: 3323752
Current size: 28244296
Opening: 47770.37.feats
Current length: 3326344
Current size: 28244296
Opening: 47770.42.feats
Current length: 3328390
Current size: 28244296
Opening: 47770.43.feats
Current length: 3330635
Current size: 28244296
Opening: 47770.44.feats
Current length: 3332895
Current size: 28244296
Opening: 47770.64.feats
Current length: 3335323
Current size: 28244296
Openin

Opening: 575605.3.feats
Current length: 3558515
Current size: 31774880
Opening: 575606.3.feats
Current length: 3560008
Current size: 31774880
Opening: 575607.3.feats
Current length: 3561505
Current size: 31774880
Opening: 585517.3.feats
Current length: 3563134
Current size: 31774880
Opening: 585520.12.feats
Current length: 3565033
Current size: 31774880
Opening: 585520.4.feats
Current length: 3567143
Current size: 31774880
Opening: 585524.3.feats
Current length: 3569099
Current size: 31774880
Opening: 585524.9.feats
Current length: 3570748
Current size: 31774880
Opening: 596325.3.feats
Current length: 3572430
Current size: 31774880
Opening: 596326.3.feats
Current length: 3573978
Current size: 31774880
Opening: 60520.3.feats
Current length: 3577070
Current size: 31774880
Opening: 60520.4.feats
Current length: 3580038
Current size: 31774880
Opening: 616990.3.feats
Current length: 3581778
Current size: 31774880
Opening: 630527.3.feats
Current length: 3584094
Current size: 31774880
Opening

In [ ]:
# USE THIS FOR FEATURES PRESENT IN MODELS
file_name = "/home/bneubert/model.features"
feature_list = json.load(open(file_name,"rb"))

In [36]:
#feature_list

In [18]:
len(feature_list)

483713

In [6]:
pangenome = [] 
coregenome = []
pangenome_sizes = []
coregenome_sizes = []
genome_sizes = []
processed_genomes = []
temp_coregenome = []
current_features = []
genomeDict = {} 
second = True # boolean used to delineate first and second genome's addition, becomes false on third genome
familyDict = {} # will contain all unique cross-genus families and their associated strains
accessoryGenome = {}

In [7]:
# NOTE: analysis assumes the genomic features are grouped together by their genomic id since they were read in by feature files


for feature in feature_list:
    
    
    # if the genome is new add genome to processed genomes
    if(feature['genome_id'] not in processed_genomes):
        # if we just switched to a new genome then
        # add the new pangenome and core genome sizes to their lists
        # also update core genome
        
        if(len(processed_genomes) > 0):
            pangenome_sizes.append(len(pangenome))
            genome_sizes.append(len(current_features))
            if not second:
                # new coregenome is simply the previous temp coregenome
                coregenome = list(temp_coregenome)
            else:
                # no longer second
                second = False
            coregenome_sizes.append(len(coregenome))
            temp_coregenome = []
            

        # add new genome to processed list
        processed_genomes.append(feature['genome_id'])
        
        #add new genome as a key in the genome dict
        genomeDict[feature['genome_id']] = {}
        
        # reset current features
        current_features = []
      
    # if the gene family has not been accounted for in the current genome's features, add it
    if(feature['pgfam_id'] not in current_features):
        current_features.append(feature['pgfam_id'])
    
    # if gene family has not been accounted for then add to pangenome
    if(feature['pgfam_id'] not in pangenome):
        pangenome.append(feature['pgfam_id'])
    
    # if first runthrough then the core genes are all genes in the first
    # genome, these are used as a baseline for future comparisons   
    if (len(processed_genomes) is 1):
        # also make sure this family has not already been accounted for
        if(feature['pgfam_id'] not in coregenome):
            coregenome.append(feature['pgfam_id'])
    
    # if we already have a core genome, the present feature is in the core genome of the past strains, and not already added
    # then include this feature in the next core genome
    elif((feature['pgfam_id'] in coregenome) and (feature['pgfam_id'] not in temp_coregenome)):
        temp_coregenome.append(feature['pgfam_id'])

        
    # update the genome dictionary
    if (feature['pgfam_id'] not in genomeDict[feature['genome_id']]):
        genomeDict[feature['genome_id']][feature['pgfam_id']] = 1
    else:
        genomeDict[feature['genome_id']][feature['pgfam_id']] += 1
        
    # if the cross genus family has not been accounted for then add it to the family dictionary
    if feature['pgfam_id'] not in familyDict:
        familyDict[feature['pgfam_id']] = [feature['genome_id']]
    # if the cross genus family has been accounted for, but the strain it came from has not been accounted for, update the
    # key's list
    elif feature['genome_id'] not in familyDict[feature['pgfam_id']]:
        familyDict[feature['pgfam_id']].append(feature['genome_id'])
        
        
# make sure to add the last genome processed to have final pangenome and coregenome sizes
pangenome_sizes.append(pangenome.__len__())
coregenome = list(temp_coregenome)
coregenome_sizes.append(coregenome.__len__())
genome_sizes.append(len(current_features))

In [8]:
len(pangenome)

2186

In [9]:
len(coregenome)

36

In [22]:
min(genome_sizes)

320

In [10]:
coregenome

[u'PGF_01960322',
 u'PGF_00065887',
 u'PGF_00064393',
 u'PGF_01625234',
 u'PGF_04345693',
 u'PGF_00056897',
 u'PGF_00007007',
 u'PGF_00063974',
 u'PGF_00049893',
 u'PGF_00033359',
 u'PGF_03609651',
 u'PGF_02346768',
 u'PGF_00020735',
 u'PGF_00024567',
 u'PGF_03026450',
 u'PGF_00006935',
 u'PGF_00422959',
 u'PGF_00008337',
 u'PGF_00033197',
 u'PGF_05463808',
 u'PGF_00420819',
 u'PGF_00420806',
 u'PGF_00016357',
 u'PGF_00024308',
 u'PGF_00427299',
 u'PGF_06522349',
 u'PGF_01435526',
 u'PGF_04521913',
 u'PGF_05171623',
 u'PGF_00013490',
 u'PGF_02179609',
 u'PGF_00772015',
 u'PGF_06162930',
 u'PGF_00037588',
 u'PGF_00069345',
 u'PGF_05500127']

In [23]:
len(processed_genomes)

848

In [24]:
unique = []
count = 0
for species in feature_list_species:
    for feature in species:
        if(feature['genome_id'] not in unique):
            unique.append(feature['genome_id'])
print(len(unique))
    

NameError: name 'feature_list_species' is not defined

In [ ]:
gone = []
for genome in feature_files:
    found = False
    for genome2 in processed_genomes:
        if(str(genome2)+'.feats' == genome):
            found = True
            break
    if(not found):
        gone.append(genome)
        print(genome)

In [ ]:
print(len(gone))

In [ ]:
feature_files

In [ ]:
#genomeDict

In [ ]:
#familyDict

In [ ]:
x = []
close = []
for family, genome in familyDict.items():
    if((len(genome)) >= (.99*len(processed_genomes)) ):
        x.append(family)
        
    if(len(genome) == (len(processed_genomes)-1)):
        close.append(family)

In [ ]:
len(x)

In [ ]:
close

In [ ]:
for genome in processed_genomes:
    if genome not in familyDict[u'PGF_01625234']:
        print(genome)
    if genome not in familyDict[u'PGF_00048782']:
        print(genome)
    if genome not in familyDict[u'PGF_01435526']:
        print(genome)
    if genome not in familyDict[u'PGF_04521913']:
        print(genome)
    if genome not in familyDict[u'PGF_01351158']:
        print(genome)
    

In [ ]:
print(len(genomeDict[unicode('1423751.3')].keys()))
print(len(genomeDict[unicode('1637506.3')].keys()))
print(len(genomeDict[unicode('60520.4')].keys()))

In [25]:
plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x = list(range(1,1507)),
    y = coregenome_sizes,
    mode = 'lines',
    name = 'core genome'
)
trace2 = go.Scatter(
    x = list(range(1,1507)),
    y = pangenome_sizes,
    mode = 'lines',
    name = 'pan genome'
)
trace3 = go.Scatter(
    x = list(range(1,1507)),
    y = genome_sizes,
    mode = 'lines',
    name = 'added genome'
)
data = [trace1, trace2,trace3]

layout = dict(title = 'Lactobacillus Analysis',
              xaxis = dict(title = 'Number of Strains Added'),
              yaxis = dict(title = 'Number of Unique Gene Families'),
              )

fig = dict(data=data, layout=layout)

plotly.offline.iplot(fig, filename='/home/bneubert/code/filtered-species-pangenome-line.html')

In [267]:
y = []
for i in range(0,len(processed_genomes)):
    y.append(0)
    
for family,genome in familyDict.items():
    y[len(genome)-1] += 1
    
    
data = [go.Bar(
            x=list(range(1,len(processed_genomes)+1)),
            y=y
    )]

layout = go.Layout(
    title='Prevalence of Protein Families in Lactobacillus Strains',
    xaxis=dict(
        title = 'Number of genomes'
        
    ),
    yaxis=dict(
        title='Number of protein families',
    ),
    
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='proximity-to-core-bar-graph')

In [295]:
for family,genome in familyDict.items():
    # if this cross-genus family is not apart of the coregenome and the cross-genus family is not unique to one strain
    # add this cross-genus family and associated strains to the accessoryGenome
    if((len(genome) < len(processed_genomes)) and (len(genome)>1)):
        accessoryGenome[family] = genome

In [296]:
full_heat = []
for i in range(0, accessoryGenome.keys().__len__()):
    temp = []
    for j in range(0,processed_genomes.__len__()):
        temp.append(0)
    full_heat.append(temp)

In [297]:
accessoryGenomeList = list(accessoryGenome.keys())

In [298]:
# Summary information

print("Number of genomes processed: " + str(len(processed_genomes)))
print("Number of unique protein families in the core genome: "+ str(len(coregenome)))
print("Number of unique protein families in the pan genome: "+ str(len(pangenome)))
print("Number of unique protein families in the accessory genome: "+ str(len(accessoryGenomeList)))
print("Average number of unique protein families in each processed genome: "+ str(sum(genome_sizes)/len(genome_sizes)))

Number of genomes processed: 1022
Number of unique protein families in the core genome: 39
Number of unique protein families in the pan genome: 51646
Number of unique protein families in the accessory genome: 35448
Average number of unique protein families in each processed genome: 2076


In [55]:
num = 0

for strain in processed_genomes:
    for family, count in genomeDict[strain].items():
        # if strain has the family and the family is in the accessory genome then the index becomes 1 in the heat indicating presence
        if(family in accessoryGenomeList):
            # family indices based upon accessory genome list, strain indices based upon processed_genomes
            full_heat[accessoryGenomeList.index(family)][num] = 1
            
    num += 1

In [56]:
#full_heat

In [57]:
trace = go.Heatmap(z=full_heat, x=list(range(1,len(processed_genomes)+1)), y=list(range(1,len(accessoryGenomeList)+1)) )

data=[trace]
            
# Set layout
layout = go.Layout(
    title= 'Lactobacillus Accessory Genome',
    xaxis=dict(
        title='Lactobacillus Strain',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Accessory Gene Family',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='/home/bneubert/Lactobacillus/Results/lactobacillus-feature-heatmap.html')

'file:///home/bneubert/Lactobacillus/Results/lactobacillus-feature-heatmap.html'

In [58]:
summary_heat = []
for i in range(0,len(accessoryGenomeList)):
    summary_heat.append([0])

In [59]:
for strain in processed_genomes:
    for family, count in genomeDict[strain].items():
        if(family in accessoryGenomeList):
            # heat of protein family at corresponding index based upon accessory genome list
            summary_heat[accessoryGenomeList.index(family)][0] += 1

In [60]:
trace = go.Heatmap(z=summary_heat, x=[], y=list(range(1,len(accessoryGenomeList)+1)) )

data=[trace]
            
# Set layout
layout = go.Layout(
    title= 'Lactobacillus Accessory Genome',
    xaxis=dict(
        title='Lactobacillus Strain',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Accessory Gene Family',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='/home/bneubert/Lactobacillus/Results/lactobacillus-summary-features-heatmap.html')

'file:///home/bneubert/Lactobacillus/Results/lactobacillus-summary-features-heatmap.html'